# От множества к пространству

Чем «пространство» отличается от «множества»? Если совсем на пальцах. Тем, что над его элементами определены операции. Соответственно, если мы просто «напишем» класс, то определим тип данных, элемент множества. А если реализуем для него операции — определим пространство.

Тяжело найти что-то, чего нет в стандартной библиотеке. А тем более во внешних...

![](https://imgs.xkcd.com/comics/python.png)

Давайте считать, что кольцо многочленов не реализовано. А ведь полезная вещь! А как его реализовать? А очень просто: [вот тут всё написано](https://docs.python.org/3.7/reference/datamodel.html#emulating-numeric-types)!

## Вначале немного про `isinstance` и `issubclass`

In [2]:
import numbers

print(
    isinstance(3.0, numbers.Number),  # проверка, что тип числовой
    isinstance(3.0, float), # успешная проверка, что тип — конкретно float
    isinstance(3, float), # неуспешная проверка, что тип — конкретно float
    isinstance(1j, numbers.Number) # проверка, что тип числовой
)

print(
    issubclass(float, float),  # проверка, что тип является подтипом (частным случаем) себя же
    issubclass(type(1j), numbers.Number), # что тип комплексного — подтип числа
    issubclass(numbers.Integral, numbers.Rational) # а тип целого — подтип рационального
)

True True False True
True True True


## Затем сам пример

In [3]:
from numbers import Number
import itertools


class PolynomialDomainError(ValueError):
    pass


class Polynomial:
    """Многочлен над полем (пока числовым, а там видно будет)"""
    
    def __init__(self, arg=0):
        """
        coefficients -- коэффициенты
        """
        if isinstance(arg, Number):
            self.coefficients = [arg]
        elif isinstance(arg, list):
            self.coefficients = arg.copy()
        elif isinstance(arg, Polynomial):
            self.coefficients = arg.coefficients.copy()
        else:
            raise PolynomialDomainError("You are trying to create polynomial from " + repr(arg))

        self.cleanup(self.coefficients)

        
    @staticmethod
    def cleanup(coefficients):
        """
        Удаление старших коэффициентов, если они близки к 0, для понижения степени
        """
        epsilon = 1e-25
        while len(coefficients) and abs(coefficients[-1]) < epsilon:
            del coefficients[-1]
        
    def __str__(self):
        return (" + ".join([
            str(c) + ("*x^" + str(i) if i > 0 else "")
            for c, i in reversed(list(zip(self.coefficients, itertools.count())))
        ])) if len(self.coefficients) else '0'

    
    def __eq__(self, other):
        """Вот тут точно полезно почитать https://wiki.python.org/moin/MultipleDispatch..."""
        if isinstance(other, Number):
            other = Polynomial(other)
        
        if isinstance(other, Polynomial):
            return self.coefficients == other.coefficients
        else:
            raise PolynomialDomainError("Can't say if Polynomial is equal to " + str(type(other)))

    
    def __lshift__(self, deg):
        return Polynomial(([0] * deg) + self.coefficients)
    
    def __add__(self, other):
        if isinstance(other, Number):
            other = Polynomial(other)
        
        sc = self.coefficients.copy()
        oc = other.coefficients.copy()
        
        # make lengths equal
        sc += [0] * (len(oc)-len(sc))
        oc += [0] * (len(sc)-len(oc))
        
        return Polynomial([
            sce + oce for sce, oce in zip(sc, oc)
        ])
        
        
    def __radd__(self, other):
        return self.__add__(other)  # Коммутативность

    
    def __neg__(self):
        return Polynomial([-c for c in self.coefficients])

    
    def __sub__(self, other):
        if isinstance(other, Number):
            other = Polynomial(other)

        return self.__add__(other.__neg__())

    
    def __rsub__(self, other):
        return self.__neg__().__add__(other)

    
    def __mul__(self, other):
        if isinstance(other, Number):
            other = Polynomial(other)
        
        c = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for sc, sci in zip(self.coefficients, itertools.count()):
            for oc, oci in zip(other.coefficients, itertools.count()):
                c[sci + oci] += sc * oc
        
        return Polynomial(c)
        
        
    def __rmul__(self, other):
        return self.__mul__(other)  # Коммутативность

    
    def __divmod__(self, other):
        if isinstance(other, Number):
            other = Polynomial(other)

        sc = Polynomial(self)
        oc = other
        d = []
        
        while len(sc.coefficients) >= len(oc.coefficients) > 0:
            c = sc.coefficients[-1] / oc.coefficients[-1]
            sc -= c * (oc << (len(sc.coefficients) - len(oc.coefficients)))
            d.append(c)
        
        return Polynomial(list(reversed(d))), sc
    

    def __floordiv__(self, other):
        return divmod(self, other)[0]

    def __mod__(self, other):
        return divmod(self, other)[1]
    

    def __rfloordiv__(self, other):
        return divmod(other, self)[0]

    def __rmod__(self, other):
        return divmod(other, self)[1]
    

    
    def __rdivmod__(self, other):
        return Polynomial(other).__divmod__(self)
    
    def __complex__(self):
        """Преобразование к комплексному числу, complex(...)"""
        if not len(self.coefficients):
            return 0j
        elif len(self.coefficients) == 1:
            return complex(self.coefficients[0])
        else:
            raise PolynomialDomainError("Can't consider higher degree polynomial as a complex")
        

    
    def __float__(self):
        """Преобразование к комплексному числу, complex(...)"""
        if not len(self.coefficients):
            return 0.0
        elif len(self.coefficients) == 1:
            return complex(self.coefficients[0])
        else:
            raise PolynomialDomainError("Can't consider higher degree polynomial as a float")
        

    def __int__(self) -> int:
        if not len(self.coefficients):
            return 0
        elif len(self.coefficients) == 1:
            return complex(self.coefficients[0])
        else:
            raise PolynomialDomainError("Can't consider higher degree polynomial as an integer")
        
        return int(self.coefficients[0])

Ну а теперь давайте посмотрим его в деле!

In [4]:
p1 = Polynomial()
p2 = Polynomial([1,2,3,4e-30])
p3 = 3

print(p1, p2)
print(p1 == p2)
print(p1 == 0, p2 == 1)
print(p2)
print(int(p3), float(p3), complex(p3))

print(p3 + p2)
print((p3 * (p3-p2) * p3) << 2)

m1 = Polynomial([1,2,3])
m2 = Polynomial([3,2,1])
m3 = Polynomial([4,5])

m12 = m1 * m2
m123 = m12 + m3
d, r = divmod(m12, m1)
print(d, r)
d = m123 // m1
r = m123 % m1
print(d, r)

print(3 // Polynomial(4))
print(3 % Polynomial([4, 5]))

0 3*x^2 + 2*x^1 + 1
False
True False
3*x^2 + 2*x^1 + 1
3 3.0 (3+0j)
3*x^2 + 2*x^1 + 4
-27*x^4 + -18*x^3 + 18*x^2 + 0*x^1 + 0
1.0*x^2 + 2.0*x^1 + 3.0 0
1.0*x^2 + 2.0*x^1 + 3.0 5.0*x^1 + 4.0
0.75
3


## Задание

Придумать и реализовать какой-нибудь свой тип данных с операциями. Вот идеи:

* числа по модулю;
* обычные числа, но с ошибками при вычислениях;
* [кватеринионы](https://en.wikipedia.org/wiki/Quaternion);
* можно матрицы (в т.ч. вектора) — они и так в NumPy есть, но всё равно же интересно;
* да тот же многочлен, но при помощи [`defaultdict`](https://docs.python.org/3.7/library/collections.html#collections.defaultdict) — ему не повредит;
* что угодно на свой вкус, над чем можно определить арифметику или её подобие.